This notebook is given for information purposes. Some lines may need to be adapted for the code to run correctly.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.collections import LineCollection
import matplotlib.colors as mpl_colors
import scipy.integrate as itp
from tqdm.notebook import tqdm

from dabry.flowfield import GyreFF
from dabry.solver_ef import SolverEFResampling
from dabry.problem import NavigationProblem

plt.rc('font', size=18)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=10)
plt.rc('mathtext', fontset='cm')
plt.rc('text', usetex=True)

In [ ]:
v_a = 1

# Kularatne 2016

x_init = np.array((0.6, 0.6))
x_target = np.array((2.4, 2.4))
bl = np.array((0.4, 0.4))
tr = np.array((2.6, 2.6))
bl2 = np.array((-1.8, -1.8))
tr2 = np.array((3, 3))

bl_large = np.array((-50, -50))
tr_large = np.array((50, 50))

gyre = GyreFF(0.5, 0.5, 2, 2, 1)

pb = NavigationProblem(gyre, x_init, x_target, v_a, bl=bl_large, tr=tr_large)

In [ ]:
#nx, ny = 101, 101w
nx, ny = 45, 45
bounds = np.array((bl2, tr2)).transpose()
xx, yy = np.meshgrid(np.linspace(*bounds[0], nx), np.linspace(*bounds[1], ny), indexing='ij')
grid_2D = np.stack((xx, yy), axis=-1)
ttt, xxx, yyy = np.meshgrid(np.linspace(0, 0, 1), np.linspace(*bounds[0], nx), np.linspace(*bounds[1], ny), indexing='ij')
grid = np.stack((np.squeeze(ttt), np.squeeze(xxx), np.squeeze(yyy)), axis=-1)
wind_dvalues = gyre.d_value_vec(grid)

In [ ]:
dvalues_norm = np.linalg.norm(wind_dvalues, axis=(-2, -1))

In [ ]:
dvalues_norm.mean()

In [ ]:
from dabry.problem import NavigationProblem

pb_unscaled = NavigationProblem.from_name('gyre')
pb = pb_unscaled.rescale()

In [ ]:
mat = -np.array(((0, 1), (-1, 0))) @ pb.model.ff.d_value(0, np.array((0.21, 0.8))).T

In [ ]:
def prepare_wind():
    fig, ax = plt.subplots(figsize=(5, 5))
    
    #nx, ny = 101, 101
    nx, ny = 45, 45
    bounds = np.array((bl2, tr2)).transpose()
    xx, yy = np.meshgrid(np.linspace(*bounds[0], nx), np.linspace(*bounds[1], ny), indexing='ij')
    ttt, xxx, yyy = np.meshgrid(np.linspace(0, 0, 1), np.linspace(*bounds[0], nx), np.linspace(*bounds[1], ny), indexing='ij')
    grid = np.stack((np.squeeze(ttt), np.squeeze(xxx), np.squeeze(yyy)), axis=-1)
    wind_values = gyre.value_vec(grid)
    
    #ax.pcolormesh(xx, yy, np.linalg.norm(wind_values, axis=-1), cmap='Greys', shading='gouraud')
    # rect = patches.Rectangle((bounds[0, 0], bounds[1, 0]), bounds[0, 1] - bounds[0, 0], bounds[1, 1] - bounds[1, 0], 
    #                          facecolor='white', alpha=0.4)
    # ax.add_patch(rect)
    n_quiv = 2
    off = 0
    ax.grid(True, color='black', alpha=0.5)
    ax.quiver(xx[off::n_quiv, off::n_quiv], yy[off::n_quiv, off::n_quiv], 
              wind_values[off::n_quiv, off::n_quiv, 0], wind_values[off::n_quiv, off::n_quiv, 1], alpha=0.6)

    # mat = -np.array(((0, 1), (-1, 0))) @ wind_dvalues[ix, iy].T
    # thetas = np.linspace(0, 2*np.pi, 100)
    # h = np.vstack((np.cos(thetas), np.sin(thetas))).T
    # r = np.einsum('ij,jk,ik->i', h, mat, h) * 5e-2

    # base_points = np.array((xx[ix, iy], yy[ix, iy])) + np.expand_dims(r, 1) * h
    # points = base_points.reshape(-1,1,2)
    # lc = LineCollection(np.concatenate([points[:-1], points[1:]], axis=1), cmap=plt.get_cmap('bwr'), linewidth=2)
    # lc.set_array(r > 0)
    # plt.gca().add_collection(lc)
    
    return fig, ax

In [ ]:
plt.hist(np.linalg.norm(wind_dvalues, axis=(-2, -1)).flatten())

In [ ]:
solver = SolverEFResampling(pb, max_depth=1, total_duration=1.6, n_costate_sectors=8000)
solver.solve()

In [ ]:
trajs = []
e_set = np.zeros((len(solver.trajs), 2))
for i_traj, traj in enumerate(solver.trajs):
    trajs.append(traj.states)
    e_set[i_traj, :] = traj.states[-1]

In [ ]:
e_set_dists = np.linalg.norm(np.vstack((e_set[0] - e_set[-1], (e_set[1:] - e_set[:-1]))), axis=-1)
print(e_set_dists.max())

In [ ]:
fig, ax = prepare_wind()

norm = mpl_colors.Normalize()
norm.autoscale([0, 0.1])

ax.plot(*np.vstack((e_set, e_set[0])).transpose(), color='black', linewidth=3.5)
points = e_set.reshape(-1,1,2)
lc = LineCollection(np.concatenate([points[:-1], points[1:]], axis=1), cmap=plt.get_cmap('jet'), linewidth=2)
lc.set_array(e_set_dists/e_set_dists.max())
plt.gca().add_collection(lc)

# ax.plot(*e_set.transpose(), linewidth=2)

for i in [5858]: #range(0, len(trajs), 5):
    #ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=2.5, color='white')
    ax.plot(*trajs[i].transpose(), linewidth=1, color='black')
    #ax.scatter(trajs[i][-1, 0], trajs[i][-1, 1], color='black', edgecolor='white', marker='D', zorder=10)

ax.scatter(*x_init, color='black', edgecolor='white', s=100, zorder=10)
ax.scatter(*x_target, color='black', edgecolor='white', s=200, marker='*', zorder=10)
circ = patches.Circle(x_target, 0.1, facecolor=(1, 0, 0, 0.5), edgecolor='red', linewidth=2)
ax.add_patch(circ)

trj = solver.trajs[5858]
for it in range(0, trj.states.shape[0], 8):

    mat = -np.array(((0, 1), (-1, 0))) @ gyre.d_value(0, trj.states[it]).T
    thetas = np.linspace(0, 2*np.pi, 100)
    h = np.vstack((np.cos(thetas), np.sin(thetas))).T
    r = np.einsum('ij,jk,ik->i', h, mat, h)

    # shifts = np.expand_dims(r, 1) * h * 5e-2
    # base_points = trj.states[it] + shifts
    # points = base_points.reshape(-1,1,2)
    # lc = LineCollection(np.concatenate([points[:-1], points[1:]], axis=1), cmap=plt.get_cmap('bwr'), linewidth=2)
    # lc.set_array(r > 0)
    # plt.gca().add_collection(lc)

    svd = np.linalg.svd(mat)
    v1 = svd.S[0] * svd.U[:, 0] * 5e-2
    v2 = svd.S[1] * svd.U[:, 1] * 5e-2
    print(svd.S)
    plt.plot([trj.states[it, 0] - v1[0], trj.states[it, 0] + v1[0]],
             [trj.states[it, 1] - v1[1], trj.states[it, 1] + v1[1]], color='red' if np.einsum('j,jk,k->', v1, mat, v1) > 0 else 'blue')
    
    plt.plot([trj.states[it, 0] - v2[0], trj.states[it, 0] + v2[0]],
             [trj.states[it, 1] - v2[1], trj.states[it, 1] + v2[1]], color='red' if np.einsum('j,jk,k->', v2, mat, v2) > 0 else 'blue')


    h = - trj.costates[it] / np.linalg.norm(trj.costates[it])
    nrm = 0.2
    ax.plot([trj.states[it, 0], trj.states[it, 0] + nrm * h[0]],
            [trj.states[it, 1], trj.states[it, 1] + nrm * h[1]], color='black')

# ax.set_xticks(np.arange(-1.5, 2.51, 1))
# ax.set_yticks(np.arange(-1.5, 2.51, 1))
# ax.set_xlim(bl2[0], tr2[0])
# ax.set_ylim(bl2[1], tr2[1])
ax.set_xlim(0.5, 2.5)
ax.set_ylim(0.5, 2.5)

#plt.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/algo_comparison/equisampling.pdf')

In [ ]:
solver2 = SolverEFResampling(pb, max_depth=20, total_duration=1.6, n_costate_sectors=50, target_radius=0.1)
solver2.solve()

In [ ]:
trajs2 = []
e_set2 = np.zeros((len(solver2.trajs), 2))

site_0 = list(solver2.sites.values())[0]
i = 0
e_set2[i, :] = site_0.traj.states[-1]
i += 1
site = site_0.next_nb[-1]
while site != site_0:
    e_set2[i, :] = site.traj.states[-1]
    trajs2.append(site.traj.states)
    i += 1
    site = site.next_nb[-1]

In [ ]:
len(trajs2)

In [ ]:
e_set_dists2 = np.linalg.norm(np.vstack((e_set2[0] - e_set2[-1], (e_set2[1:] - e_set2[:-1]))), axis=-1)
print(e_set_dists2.max())

In [ ]:
fig, ax = plt.subplots()
ax.hist(e_set_dists, density=True, bins=np.linspace(0, 0.1, 25), histtype='step', log=False, label='Equisampling')
ax.hist(e_set_dists2, density=True, bins=np.linspace(0, 0.1, 25), histtype='step', log=False, label='In-depth sampling')
ax.set_xlabel('Distance between neighbors (-)')
ax.set_ylabel('Occurences (-)')
ax.set_ylim(0, 50)
ax.text(0.0006, 42, '200→', rotation=90, color='C0', fontsize=14)
ax.text(0.084, 1.5, '←', rotation=90, color='C0', fontsize=14)
ax.text(0.092, 1.5, '←', rotation=90, color='C0', fontsize=14)
ax.tick_params(direction='in')
ax.grid(True, linestyle='--', alpha=0.5)
plt.legend()
#plt.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/algo_comparison/hist.pdf')

In [ ]:
tot_dur = [1.6, 2, 2.4, 2.8, 3.2, 3.6]
cpu_times = [2.11, 4.1, 5.74, 9.29, 12.74, 16.44]
n_trajs = [259, 403, 609, 920, 1212, 1228]
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(14, 6))
ax[0].plot(tot_dur, cpu_times)
ax[1].plot(tot_dur, n_trajs)
ax[2].plot(tot_dur, np.array(cpu_times)/np.array(n_trajs))

In [ ]:
fig, ax = prepare_wind()

norm = mpl_colors.Normalize()
norm.autoscale([0, 0.1])

ax.plot(*np.vstack((e_set, e_set[0])).transpose(), color='black', linewidth=3.5)
points = e_set2.reshape(-1,1,2)
lc = LineCollection(np.concatenate([points[:-1], points[1:]], axis=1), cmap=plt.get_cmap('jet'), linewidth=2)
lc.set_array(e_set_dists2/e_set_dists2.max())
plt.gca().add_collection(lc)



for i in [302]:#range(0, len(trajs), 5):
    #ax.plot(trajs[i, :, 0], trajs[i, :, 1], linewidth=2.5, color='white')
    ax.plot(*trajs2[i].transpose(), linewidth=1, color='black')
    #ax.scatter(trajs2[i][-1, 0], trajs2[i][-1, 1], color='black', edgecolor='white', marker='D', zorder=10)

ax.scatter(*x_init, color='black', edgecolor='white', s=100, zorder=10)
ax.scatter(*x_target, color='black', edgecolor='white', s=200, marker='*', zorder=2)
circ = patches.Circle(x_target, 0.1, facecolor=(1, 0, 0, 0.5), edgecolor='red', linewidth=2)
ax.add_patch(circ)

ax.set_xticks(np.arange(-1.5, 2.51, 1))
ax.set_yticks(np.arange(-1.5, 2.51, 1))
ax.set_xlim(bl2[0], tr2[0])
ax.set_ylim(bl2[1], tr2[1])

#plt.savefig('/home/bastien/Documents/Manuscript/manuscript/time_optimality/plot_scripts/algo_comparison/indepth.pdf')

In [ ]:
traj = trajs[0]
ztraj = np.column_stack((np.zeros(traj.shape[0]), traj))

In [ ]:
np.linalg.norm(gyre.d_value_vec(ztraj), axis=(-2, -1))

In [ ]:
solverA = SolverEFResampling(pb, max_depth=1, total_duration=1.2, n_costate_sectors=2000)
solverA.solve()

In [ ]:
trajs = []
e_set = np.zeros((len(solverA.trajs), 2))
for i_traj, traj in enumerate(solverA.trajs):
    trajs.append(traj.states)
    e_set[i_traj, :] = traj.states[-1]

In [ ]:
e_set_dists = np.linalg.norm(np.vstack((e_set[0] - e_set[-1], (e_set[1:] - e_set[:-1]))), axis=-1)
print(e_set_dists.max())

| param             |         |         |         |
| ------------      | ---     |  ---    | ----    |
| total_time        | 0.8     |   1.2   | 1.6     |
| equi              |         |         |         |
| n_trajs           | 500     |  2000   | 8000    |
| e_dist_max        | 0.1113  | 0.1013  | 0.1126  |
| in depth          |         |         |         |
| n_trajs           | 128     | 264     |  501    |
| e_dist_max        | 0.09888 | 0.09931 | 0.09987 |

In [ ]:
n_trajs = [128, 264, 501]
total_time = [0.8, 1.2, 1.6]

In [ ]:
plt.plot(total_time, np.log(n_trajs))

In [ ]:
solverB = SolverEFResampling(pb, max_depth=20, total_duration=1.2, n_costate_sectors=50, target_radius=0.1)
solverB.solve()

In [ ]:
trajs2 = []
e_set2 = np.zeros((len(solverB.trajs), 2))

site_0 = list(solverB.sites.values())[0]
i = 0
e_set2[i, :] = site_0.traj.states[-1]
i += 1
site = site_0.next_nb[-1]
while site != site_0:
    e_set2[i, :] = site.traj.states[-1]
    trajs2.append(site.traj.states)
    i += 1
    site = site.next_nb[-1]

In [ ]:
len(trajs2)

In [ ]:
e_set_dists2 = np.linalg.norm(np.vstack((e_set2[0] - e_set2[-1], (e_set2[1:] - e_set2[:-1]))), axis=-1)
print(e_set_dists2.max())

In [ ]:
2 * np.pi * 1/0.1 * np.exp(3.0111*1.6) 